### Imports

In [1]:
from scenarionet import read_dataset_summary, read_scenario
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image

### Nuscenes

##### Read dataset summary

In [2]:
read_dataset_summary?

Signature: read_dataset_summary(dataset_path)
Docstring:
Read the dataset and return the metadata of each scenario in this dataset.

Args:
    dataset_path: the path to the root folder of your dataset.

Returns:
    A tuple of three elements:
    1) the summary dict mapping from scenario ID to its metadata,
    2) the list of all scenarios IDs, and
    3) a dict mapping from scenario IDs to the folder that hosts their files.
File:      /data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/scenarionet/scenarionet/common_utils.py
Type:      function

In [3]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/nuscenes_mini_converted_10/')

In [4]:
mapping

{'sd_nuscenes_v1.0-mini_scene-0061.pkl': 'nuscenes_mini_converted_0',
 'sd_nuscenes_v1.0-mini_scene-0103.pkl': 'nuscenes_mini_converted_1',
 'sd_nuscenes_v1.0-mini_scene-0553.pkl': 'nuscenes_mini_converted_2',
 'sd_nuscenes_v1.0-mini_scene-0655.pkl': 'nuscenes_mini_converted_3',
 'sd_nuscenes_v1.0-mini_scene-0757.pkl': 'nuscenes_mini_converted_4',
 'sd_nuscenes_v1.0-mini_scene-0796.pkl': 'nuscenes_mini_converted_5',
 'sd_nuscenes_v1.0-mini_scene-0916.pkl': 'nuscenes_mini_converted_6',
 'sd_nuscenes_v1.0-mini_scene-1077.pkl': 'nuscenes_mini_converted_7',
 'sd_nuscenes_v1.0-mini_scene-1094.pkl': 'nuscenes_mini_converted_8',
 'sd_nuscenes_v1.0-mini_scene-1100.pkl': 'nuscenes_mini_converted_9'}

In [7]:
scenario_ids

['sd_nuscenes_v1.0-mini_scene-0061.pkl',
 'sd_nuscenes_v1.0-mini_scene-0103.pkl',
 'sd_nuscenes_v1.0-mini_scene-0553.pkl',
 'sd_nuscenes_v1.0-mini_scene-0655.pkl',
 'sd_nuscenes_v1.0-mini_scene-0757.pkl',
 'sd_nuscenes_v1.0-mini_scene-0796.pkl',
 'sd_nuscenes_v1.0-mini_scene-0916.pkl',
 'sd_nuscenes_v1.0-mini_scene-1077.pkl',
 'sd_nuscenes_v1.0-mini_scene-1094.pkl',
 'sd_nuscenes_v1.0-mini_scene-1100.pkl']

In [5]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0061.pkl'].keys()

dict_keys(['dataset', 'metadrive_processed', 'map', 'date', 'coordinate', 'id', 'scenario_id', 'sample_rate', 'ts', 'sdc_id', 'object_summary', 'number_summary'])

In [6]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0061.pkl']['sample_rate']

0.1

In [19]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0061.pkl']['ts']

[0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2000000000000002,
 1.3,
 1.4000000000000001,
 1.5,
 1.6,
 1.7000000000000002,
 1.8,
 1.9000000000000001,
 2.0,
 2.1,
 2.2,
 2.3000000000000003,
 2.4000000000000004,
 2.5,
 2.6,
 2.7,
 2.8000000000000003,
 2.9000000000000004,
 3.0,
 3.1,
 3.2,
 3.3000000000000003,
 3.4000000000000004,
 3.5,
 3.6,
 3.7,
 3.8000000000000003,
 3.9000000000000004,
 4.0,
 4.1000000000000005,
 4.2,
 4.3,
 4.4,
 4.5,
 4.6000000000000005,
 4.7,
 4.800000000000001,
 4.9,
 5.0,
 5.1000000000000005,
 5.2,
 5.300000000000001,
 5.4,
 5.5,
 5.6000000000000005,
 5.7,
 5.800000000000001,
 5.9,
 6.0,
 6.1000000000000005,
 6.2,
 6.300000000000001,
 6.4,
 6.5,
 6.6000000000000005,
 6.7,
 6.800000000000001,
 6.9,
 7.0,
 7.1000000000000005,
 7.2,
 7.300000000000001,
 7.4,
 7.5,
 7.6000000000000005,
 7.7,
 7.800000000000001,
 7.9,
 8.0,
 8.1,
 8.200000000000001,
 8.3,
 8.4,
 8.5,
 8.6,
 8.700000000000001,


In [10]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0061.pkl']['coordinate']

'right-handed'

In [11]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-1100.pkl']['sdc_id']

'ego'

In [12]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-1100.pkl']['id']

'scene-1100'

In [13]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0061.pkl']['number_summary']

{'num_objects': 226,
 'object_types': ('TRAFFIC_BARRIER',
  'VEHICLE',
  'TRAFFIC_CONE',
  'CYCLIST',
  'PEDESTRIAN'),
 'num_objects_each_type': {'PEDESTRIAN': 60,
  'TRAFFIC_BARRIER': 65,
  'TRAFFIC_CONE': 71,
  'VEHICLE': 24,
  'CYCLIST': 6},
 'num_moving_objects': 64,
 'num_moving_objects_each_type': defaultdict(int,
             {'PEDESTRIAN': 45,
              'VEHICLE': 13,
              'CYCLIST': 2,
              'TRAFFIC_BARRIER': 2,
              'TRAFFIC_CONE': 2}),
 'num_traffic_lights': 0,
 'num_traffic_light_types': (),
 'num_traffic_light_each_step': {},
 'num_map_features': 2300,
 'map_height_diff': -inf}

In [14]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0553.pkl']['number_summary']

{'num_objects': 57,
 'object_types': ('CYCLIST', 'TRAFFIC_BARRIER', 'PEDESTRIAN', 'VEHICLE'),
 'num_objects_each_type': {'PEDESTRIAN': 12,
  'TRAFFIC_BARRIER': 16,
  'VEHICLE': 28,
  'CYCLIST': 1},
 'num_moving_objects': 26,
 'num_moving_objects_each_type': defaultdict(int,
             {'PEDESTRIAN': 11, 'VEHICLE': 14, 'CYCLIST': 1}),
 'num_traffic_lights': 0,
 'num_traffic_light_types': (),
 'num_traffic_light_each_step': {},
 'num_map_features': 1872,
 'map_height_diff': -inf}

In [15]:
dataset_summary['sd_nuscenes_v1.0-mini_scene-0553.pkl']['object_summary']

{'b61b72a5b5ad482e9bc94c203f75b8d4': {'type': 'PEDESTRIAN',
  'object_id': 'b61b72a5b5ad482e9bc94c203f75b8d4',
  'track_length': 201,
  'moving_distance': 24.351477363755134,
  'valid_length': 201,
  'continuous_valid_length': 201},
 'b4db00c7834047799104363c10eca351': {'type': 'TRAFFIC_BARRIER',
  'object_id': 'b4db00c7834047799104363c10eca351',
  'track_length': 201,
  'moving_distance': 0.04441421356223234,
  'valid_length': 196,
  'continuous_valid_length': 196},
 '07bf3d834d04482e8f90e3756358250e': {'type': 'TRAFFIC_BARRIER',
  'object_id': '07bf3d834d04482e8f90e3756358250e',
  'track_length': 201,
  'moving_distance': 0.0,
  'valid_length': 201,
  'continuous_valid_length': 201},
 '01a9922c1b7d457cb37880b8de3b7093': {'type': 'TRAFFIC_BARRIER',
  'object_id': '01a9922c1b7d457cb37880b8de3b7093',
  'track_length': 201,
  'moving_distance': 0.0,
  'valid_length': 201,
  'continuous_valid_length': 201},
 '94e0b9b077774f18a74f53c5305a368d': {'type': 'PEDESTRIAN',
  'object_id': '94e0b9

In [16]:
read_scenario?

Signature: read_scenario(dataset_path, mapping, scenario_file_name)
Docstring:
Read a scenario pkl file and return the Scenario Description instance.

Args:
    dataset_path: the path to the root folder of your dataset.
    mapping: the dict mapping return from read_dataset_summary.
    scenario_file_name: the file name to a scenario file, should end with `.pkl`.

Returns:
    The Scenario Description instance of that scenario.
File:      /data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/scenarionet/scenarionet/common_utils.py
Type:      function

In [17]:
mapping

{'sd_nuscenes_v1.0-mini_scene-0061.pkl': 'nuscenes_mini_converted_0',
 'sd_nuscenes_v1.0-mini_scene-0103.pkl': 'nuscenes_mini_converted_1',
 'sd_nuscenes_v1.0-mini_scene-0553.pkl': 'nuscenes_mini_converted_2',
 'sd_nuscenes_v1.0-mini_scene-0655.pkl': 'nuscenes_mini_converted_3',
 'sd_nuscenes_v1.0-mini_scene-0757.pkl': 'nuscenes_mini_converted_4',
 'sd_nuscenes_v1.0-mini_scene-0796.pkl': 'nuscenes_mini_converted_5',
 'sd_nuscenes_v1.0-mini_scene-0916.pkl': 'nuscenes_mini_converted_6',
 'sd_nuscenes_v1.0-mini_scene-1077.pkl': 'nuscenes_mini_converted_7',
 'sd_nuscenes_v1.0-mini_scene-1094.pkl': 'nuscenes_mini_converted_8',
 'sd_nuscenes_v1.0-mini_scene-1100.pkl': 'nuscenes_mini_converted_9'}

In [21]:
scenario = read_scenario(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/nuscenes_mini_converted_10/', mapping=mapping, scenario_file_name='sd_nuscenes_v1.0-mini_scene-0061.pkl')

###### 'scenario' important keys : 'length', 'metadata', 'tracks', 'map_features'

In [66]:
scenario.keys()

dict_keys(['id', 'version', 'length', 'metadata', 'tracks', 'dynamic_map_states', 'map_features'])

In [203]:
scenario['length']

191

###### 'metadata' key has a dict value which is the same as dataset_summary
###### 'metadata' important keys : 'ts', 'object_summary', 'number_summary'

In [80]:
scenario['metadata'].keys()

dict_keys(['dataset', 'metadrive_processed', 'map', 'date', 'coordinate', 'id', 'scenario_id', 'sample_rate', 'ts', 'sdc_id', 'object_summary', 'number_summary'])

In [174]:
scenario['metadata']['ts'].shape

(191,)

In [124]:
scenario['metadata']['object_summary']

{'1511e02cbd80490299371149ed7eb269': {'type': 'PEDESTRIAN',
  'object_id': '1511e02cbd80490299371149ed7eb269',
  'track_length': 191,
  'moving_distance': 12.02638508755264,
  'valid_length': 96,
  'continuous_valid_length': 96},
 '038a951ededf4efeb34a4ab28508d9dc': {'type': 'TRAFFIC_BARRIER',
  'object_id': '038a951ededf4efeb34a4ab28508d9dc',
  'track_length': 191,
  'moving_distance': 0.0,
  'valid_length': 66,
  'continuous_valid_length': 66},
 '1e775890010e4d97b74ca1d06441d023': {'type': 'PEDESTRIAN',
  'object_id': '1e775890010e4d97b74ca1d06441d023',
  'track_length': 191,
  'moving_distance': 0.3219775634659703,
  'valid_length': 131,
  'continuous_valid_length': 131},
 '03e81d6b45c34d56bc1a886919722f7f': {'type': 'TRAFFIC_BARRIER',
  'object_id': '03e81d6b45c34d56bc1a886919722f7f',
  'track_length': 191,
  'moving_distance': 0.0,
  'valid_length': 156,
  'continuous_valid_length': 156},
 '59cfdb95669549b580058fc56d27b132': {'type': 'PEDESTRIAN',
  'object_id': '59cfdb95669549b58

In [104]:
scenario['metadata']['number_summary']

{'num_objects': 226,
 'object_types': {'CYCLIST',
  'PEDESTRIAN',
  'TRAFFIC_BARRIER',
  'TRAFFIC_CONE',
  'VEHICLE'},
 'num_objects_each_type': {'PEDESTRIAN': 60,
  'TRAFFIC_BARRIER': 65,
  'TRAFFIC_CONE': 71,
  'VEHICLE': 24,
  'CYCLIST': 6},
 'num_moving_objects': 64,
 'num_moving_objects_each_type': defaultdict(int,
             {'PEDESTRIAN': 45,
              'VEHICLE': 13,
              'CYCLIST': 2,
              'TRAFFIC_BARRIER': 2,
              'TRAFFIC_CONE': 2}),
 'num_traffic_lights': 0,
 'num_traffic_light_types': set(),
 'num_traffic_light_each_step': {},
 'num_map_features': 2300,
 'map_height_diff': -inf}

###### scenario 'track' key

###### ['tracks']['(id)'] important keys : 'type', 'state'

In [176]:
len(scenario['tracks'].keys())

226

###### len(scenario['tracks'].keys()) == to the number of objects in the scenario

In [150]:
len(scenario['tracks'].keys())

226

In [151]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269'].keys()

dict_keys(['type', 'state', 'metadata'])

In [152]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['type']

'PEDESTRIAN'

In [153]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['metadata']

{'track_length': 191,
 'type': 'human.pedestrian.adult',
 'object_id': '1511e02cbd80490299371149ed7eb269',
 'original_id': '1511e02cbd80490299371149ed7eb269'}

In [154]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state'].keys()

dict_keys(['position', 'heading', 'velocity', 'valid', 'length', 'width', 'height'])

In [180]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['position'].shape

(191, 3)

In [184]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['heading'].shape

(191,)

In [186]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['velocity'].shape

(191, 2)

In [4]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['valid'].shape

(191,)

In [188]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['length'].shape

(191, 1)

In [190]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['width'].shape

(191, 1)

In [201]:
type(scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state']['height'])

numpy.ndarray

##### Agent features 
Numpy array file representing per scenario of shape (num_agents, seq_length, dim) where dim is an array representing [valid, x, y, h, Vx, Vy, W, L] and seq_length is the length of the scenario and more precisely the length of the ts array.'

In [196]:
scenario['tracks'].keys()

dict_keys(['1511e02cbd80490299371149ed7eb269', '038a951ededf4efeb34a4ab28508d9dc', '1e775890010e4d97b74ca1d06441d023', '03e81d6b45c34d56bc1a886919722f7f', '59cfdb95669549b580058fc56d27b132', 'bbd6c47a20e34276bc6f2cfbc6ba6205', '91d424135a6344aea4b0dde64a953b19', '9a433619baf44e41901d180558f263a0', '7179ea3c6d2e42b8b974aabbaabab0f3', 'eb3da8843f614a90a48ddb6ee392c0c0', 'dad63b6319f2479fab3a8f5f75d6049b', 'ac2622bcaa704aa4993efdfde06a4235', '58ce00fe2eb547acaa7f792ff2b11742', 'c38e6667f26a4fb5a6b2f2ce9a61ad5b', '1056bca4aedb40a2b2451c5356f014c9', '100427ca2fab4537b068582d5525495a', 'd52c3540734344a9ab714948537556be', 'd76647cebfc04503905d477d62edbb7c', 'ce995b52d7204ee8b4a6ed0b2b0ebd3e', '3377938bacc3493d87cf5a91d961de56', 'b77d6bc70844489da846d3a938a48e80', 'db5d2c1232154b5f83270c510d227720', 'f11f865bc5c74e52ab78218e45b8b40a', '4b77b9d062f547a5a3cfe21ac661ea9a', 'e030ecead0154a5fb2a3e1646467780d', 'ec2c1cc2410a4e259aa9c12756e1d6ec', 'a87f4eb5c4b349cc8719e44a4dc0fcba', '36f5c9f6981d4fbf

In [199]:
scenario['tracks']['1511e02cbd80490299371149ed7eb269']['state'].keys()

dict_keys(['position', 'heading', 'velocity', 'valid', 'length', 'width', 'height'])

In [2]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')


In [3]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')

list_seq_length, list_num_agents = [], []
for sc in scenario_ids:
    scenario = read_scenario(dataset_path='/data/tii/data/nuscenes_trainval_pkl/', mapping=mapping, scenario_file_name=sc)
    seq_length = scenario['length']
    
    list_seq_length.append(seq_length)
    print('seq_length: ', seq_length)
    
    #print('shape of ts: ', scenario['metadata']['ts'].shape)

    scenario_list = []
    for agent in scenario['tracks'].keys():
        if scenario['tracks'][agent]['type'] in ['PEDESTRIAN', 'CYCLIST', 'VEHICLE']:
            state_dict = scenario['tracks'][agent]['state']
            
            #ts = scenario['metadata']['ts']  #-- array of shape (seq_length,)
            vd = state_dict['valid']         #-- array of shape (seq_length,) (either 0. or 1.)
            p  = state_dict['position']      #-- array of shape (seq_length, 3)
            hd = state_dict['heading']       #-- array of shape (seq_length,)
            v  = state_dict['velocity']      #-- array of shape (seq_length, 2)
            h  = state_dict['height']        #-- array of shape (seq_length, 1)
            w  = state_dict['width']         #-- array of shape (seq_length, 1)
            l  = state_dict['length']        #-- array of shape (seq_length, 1)
            
            agent_list = []
            for s in range(seq_length):
                # [Nuscenes paper] Data annotation. ... a cuboid modeled as x, y, z, width, length, height
                agent_list.append([vd[s], p[s][0], p[s][1], hd[s], v[s][0], v[s][1], w[s][0], l[s][0]])
                #agent_list.append([vd[s], ts[s], p[s][0], p[s][1], p[s][2], hd[s], v[s][0], v[s][1], h[s][0], w[s][0], l[s][0]])
            
            scenario_list.append(agent_list)
            
    list_num_agents.append(len(scenario_list))

    scenario_npy = np.array(scenario_list)
    print('before slicing', scenario_npy.shape)
    # truncate the numpy array to the min seq_length: 156 to have a uniform shape in dim 1 and 2
    scenario_npy = scenario_npy[:, :156, :]
    print('after slicing', scenario_npy.shape)
    print(scenario_npy.shape)
    print(type(scenario_npy))

    dir_path = '/data/tii/data/nuscenes_trainval_npy2/'
    os.makedirs(dir_path, exist_ok=True)
    full_path = os.path.join(dir_path, sc)
    new_file_path = full_path.replace('.pkl', '.npy')
    np.save(new_file_path, scenario_npy)
    print(f'Scenario {sc} saved !')

print('max seq_length: ', max(list_seq_length))
print('min seq_length: ', min(list_seq_length))
print('max number of agents: ', max(list_num_agents))
print('min number of agents: ', min(list_num_agents))


seq_length:  196
before slicing (20, 196, 8)
after slicing (20, 156, 8)
(20, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0001.pkl saved !
seq_length:  196
before slicing (42, 196, 8)
after slicing (42, 156, 8)
(42, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0002.pkl saved !
seq_length:  196
before slicing (55, 196, 8)
after slicing (55, 156, 8)
(55, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0003.pkl saved !
seq_length:  196
before slicing (52, 196, 8)
after slicing (52, 156, 8)
(52, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0004.pkl saved !
seq_length:  191
before slicing (24, 191, 8)
after slicing (24, 156, 8)
(24, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0005.pkl saved !
seq_length:  196
before slicing (32, 196, 8)
after slicing (32, 156, 8)
(32, 156, 8)
<class 'numpy.ndarray'>
Scenario sd_nuscenes_v1.0-trainval_scene-0006.pkl saved 

In [ ]:
dir_path = '/data/tii/data/nuscenes_trainval_npy'
for l in os.listdir(dir_path):
    file_path = os.path.join(dir_path, l)
    arr = np.load(file_path)
    for ag in range(10):
        print('W:', arr[ag, :, -2])
        print('L:', arr[ag, :, -1])
    break

##### Map features

###### Visualize map feature types through all scenarios in nuscenes trainval and mini: ['ROAD_LINE_SOLID_SINGLE_WHITE', 'ROAD_LINE_BROKEN_SINGLE_WHITE', 'ROAD_LINE_SOLID_SINGLE_YELLOW', 'CROSSWALK', 'ROAD_EDGE_SIDEWALK', 'LANE_SURFACE_STREET', 'LANE_SURFACE_UNSTRUCTURE']


In [ ]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')

l_type = []
for sc in scenario_ids:
    scenario = read_scenario(dataset_path='/data/tii/data/nuscenes_trainval_pkl/', mapping=mapping, scenario_file_name=sc)
    print('scenario: ', sc)
    print(len(scenario['map_features'].keys()))

    for k in scenario['map_features'].keys():
        ft_type = scenario['map_features'][k]['type']
        #print(k)
        if ft_type not in l_type:
            l_type.append(ft_type)
    
print('map feature types through all scenarios in nuscenes trainval:', l_type)

In [ ]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/nuscenes_mini_converted_10/')

scenario = read_scenario(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/nuscenes_mini_converted_10/', mapping=mapping, scenario_file_name='sd_nuscenes_v1.0-mini_scene-0103.pkl')

tracks = scenario['tracks']
num_agents = len(tracks.keys())
colors = cm.rainbow(np.linspace(0, 1, num_agents))

plt.figure()

for idx, (id, track) in enumerate(tracks.items()):
    print(id)
    xyz = track['state']['position']  #-- array of shape (seq_length, 3)
    print(xyz.shape)
    
    # Filter out invalid points
    valid_points = xyz[(xyz[:, 0] != 0.0) & (xyz[:, 1] != 0.0)]
    
    if valid_points.shape[0] > 0:
        plt.plot(valid_points[:, 0], valid_points[:, 1], color=colors[idx], label=f'Agent {id}')

    

# Remove axes
#plt.axis('off')
output_file_path = './tracks_sd_nuscenes_v1.0-mini_scene-0103.png'
plt.savefig(output_file_path, format='png')

In [ ]:
# Initialize the plot
# only polylines
plt.figure(figsize=(10, 10))

for k in scenario['map_features'].keys():
    if scenario['map_features'][k]['type'] in ['CROSSWALK', 'ROAD_EDGE_SIDEWALK']:
        for kk in scenario['map_features'][k].keys():
            if kk in 'polyline':
                polyline_arr = scenario['map_features'][k][kk]
                # Plot the polyline
                plt.plot(polyline_arr[:, 0], polyline_arr[:, 1])
            elif kk == 'polygon':
                polygon_arr = scenario['map_features'][k][kk]
                # Plot the polygon
                plt.plot(polygon_arr[:, 0], polygon_arr[:, 1])
            
# Remove axes
plt.axis('off')
output_file_path = './PED.png'
plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

In [ ]:
# Initialize the plot
# only polylines
plt.figure(figsize=(10, 10))

for k in scenario['map_features'].keys():
    if scenario['map_features'][k]['type'] in ['LANE_SURFACE_STREET', 'LANE_SURFACE_UNSTRUCTURE', 'ROAD_LINE_SOLID_SINGLE_YELLOW', 'ROAD_LINE_BROKEN_SINGLE_WHITE']:
        for kk in scenario['map_features'][k].keys():
            if kk == 'polyline':
                polyline_arr = scenario['map_features'][k][kk]
                # Plot the polyline
                plt.plot(polyline_arr[:, 0], polyline_arr[:, 1])
            elif kk == 'polygon':
                polygon_arr = scenario['map_features'][k][kk]
                # Plot the polygon
                plt.plot(polygon_arr[:, 0], polygon_arr[:, 1])
            
# Remove axes
plt.axis('off')
output_file_path = './VEH_CYC.png'
plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

In [ ]:
# Initialize the plot
# only polylines
plt.figure(figsize=(100, 100))

for k in scenario['map_features'].keys():
    if scenario['map_features'][k]['type'] in ['ROAD_LINE_BROKEN_SINGLE_WHITE', 'ROAD_LINE_SOLID_SINGLE_YELLOW', 'LANE_SURFACE_STREET', 'LANE_SURFACE_UNSTRUCTURE']:
        for kk in scenario['map_features'][k].keys():
            if kk == 'polyline':
                polyline_arr = scenario['map_features'][k][kk]
                # Plot the polyline
                plt.plot(polyline_arr[:, 0], polyline_arr[:, 1])
            elif kk == 'polygon':
                polygon_arr = scenario['map_features'][k][kk]
                # Plot the polygon
                plt.plot(polygon_arr[:, 0], polygon_arr[:, 1])

tracks = scenario['tracks']
#num_agents = len(tracks.keys())
#colors = cm.rainbow(np.linspace(0, 1, num_agents))


for idx, (id, track) in enumerate(tracks.items()):
    #print(id)
    #-- ('PEDESTRIAN', 'CYCLIST', 'TRAFFIC_CONE', 'TRAFFIC_BARRIER', 'VEHICLE')

    if track['type'] in ['CYCLIST', 'VEHICLE']:
        xyz = track['state']['position']  #-- array of shape (seq_length, 3)
        #print(xyz.shape)
        
        # Filter out invalid points
        valid_points = xyz[(xyz[:, 0] != 0.0) & (xyz[:, 1] != 0.0)]
        
        if valid_points.shape[0] > 0:
            #plt.plot(valid_points[:, 0], valid_points[:, 1], color=colors[idx], label=f'Agent {id}')
            plt.plot(valid_points[:, 0], valid_points[:, 1], color='k', label=f'Agent {id}')
        

# Remove axes
plt.axis('off')
output_file_path = './VEH_CYC_TRAJ.png'
plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

In [ ]:
# Initialize the plot
# only polylines
plt.figure(figsize=(100, 100))

for k in scenario['map_features'].keys():
    if scenario['map_features'][k]['type'] in ['CROSSWALK', 'ROAD_EDGE_SIDEWALK']:
        for kk in scenario['map_features'][k].keys():
            if kk == 'polyline':
                polyline_arr = scenario['map_features'][k][kk]
                # Plot the polyline
                plt.plot(polyline_arr[:, 0], polyline_arr[:, 1])
            elif kk == 'polygon':
                polygon_arr = scenario['map_features'][k][kk]
                # Plot the polygon
                plt.plot(polygon_arr[:, 0], polygon_arr[:, 1])

tracks = scenario['tracks']
#num_agents = len(tracks.keys())
#colors = cm.rainbow(np.linspace(0, 1, num_agents))


for idx, (id, track) in enumerate(tracks.items()):
    #print(id)
    #-- ('PEDESTRIAN', 'CYCLIST', 'TRAFFIC_CONE', 'TRAFFIC_BARRIER', 'VEHICLE')

    if track['type'] == 'PEDESTRIAN':
        xyz = track['state']['position']  #-- array of shape (seq_length, 3)
        #print(xyz.shape)
        
        # Filter out invalid points
        valid_points = xyz[(xyz[:, 0] != 0.0) & (xyz[:, 1] != 0.0)]
        
        if valid_points.shape[0] > 0:
            #plt.plot(valid_points[:, 0], valid_points[:, 1], color=colors[idx], label=f'Agent {id}')
            plt.plot(valid_points[:, 0], valid_points[:, 1], color='k', label=f'Agent {id}')
        

# Remove axes
plt.axis('off')
output_file_path = './PED_ TRAJ.png'
plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

###### Map features to png files

In [ ]:
# Define the input and output directories
input_dir = '/data/tii/data/nuscenes_maps/nuscenes_trainval_maps_png'  

'''
# Process each file in the input directory
for filename in os.listdir(input_dir):
        # Open the image
        img_path = os.path.join(input_dir, filename)
        image = Image.open(img_path)

        # Convert RGBA to RGB
        rgb_image = image.convert('RGB')

        # Convert to NumPy array for cropping
        np_image = np.array(rgb_image)

        # Crop 1 pixel from height and width
        cropped_image = np_image[:-1, :-1, :]

        # Convert back to a PIL image
        cropped_pil_image = Image.fromarray(cropped_image)

        # Save the processed image to the output directory
        output_path = os.path.join(input_dir, filename)
        cropped_pil_image.save(output_path)
        print('Cropped array shape:', cropped_image.shape)
        print('Cropped array values in (128, 100):', cropped_image[128][100])
        print(f"Processed and saved {filename}")

print("All images processed.")
#'''

In [ ]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')

for sc in scenario_ids:
    plt.figure(figsize=(2.87, 2.87))
    scenario = read_scenario(dataset_path='/data/tii/data/nuscenes_trainval_pkl/', mapping=mapping, scenario_file_name=sc) 

    for k in scenario['map_features'].keys():
        if scenario['map_features'][k]['type'] in ['ROAD_LINE_BROKEN_SINGLE_WHITE', 'ROAD_LINE_SOLID_SINGLE_YELLOW', 'LANE_SURFACE_STREET', 'LANE_SURFACE_UNSTRUCTURE']:
            for kk in scenario['map_features'][k].keys():
                if kk in ['polyline', 'polygon']:
                    poly_arr = scenario['map_features'][k][kk]
                    # Plot the polyline/polygon
                    plt.plot(poly_arr[:, 0], poly_arr[:, 1], color='b')
        elif scenario['map_features'][k]['type'] in ['ROAD_EDGE_SIDEWALK', 'CROSSWALK']:
            for kk in scenario['map_features'][k].keys():
                if kk in ['polyline', 'polygon']:
                    poly_arr = scenario['map_features'][k][kk]
                    # Plot the polyline/polygon
                    plt.plot(poly_arr[:, 0], poly_arr[:, 1], color='g')
        elif scenario['map_features'][k]['type'] == 'ROAD_LINE_SOLID_SINGLE_WHITE':
            for kk in scenario['map_features'][k].keys():
                if kk in ['polyline', 'polygon']:
                    poly_arr = scenario['map_features'][k][kk]
                    # Plot the polyline/polygon
                    plt.plot(poly_arr[:, 0], poly_arr[:, 1], color='r')

                
    # Remove axes
    plt.axis('off')
    
    # Ensure the layout is correct before saving
    plt.tight_layout()
    
    dir_path = '/data/tii/data/nuscenes_maps/nuscenes_trainval_maps_png3/'
    os.makedirs(dir_path, exist_ok=True)
    full_path = os.path.join(dir_path, sc)
    new_file_path = full_path.replace('.pkl', '.png')
    
    #print("Figure size before saving:", fig.get_size_inches())
    plt.savefig(new_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

    print(f'Map of the scenario {sc} saved in png format!')
    plt.close()



In [ ]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')

for sc in scenario_ids:
    plt.figure(figsize=(2.87, 2.87))
    scenario = read_scenario(dataset_path='/data/tii/data/nuscenes_trainval_pkl/', mapping=mapping, scenario_file_name=sc) 

    for k in scenario['map_features'].keys():
        if scenario['map_features'][k]['type'] != 'ROAD_LINE_SOLID_SINGLE_WHITE':
            for kk in scenario['map_features'][k].keys():
                if kk in ['polyline', 'polygon']:
                    poly_arr = scenario['map_features'][k][kk]
                    # Plot the polyline/polygon
                    plt.plot(poly_arr[:, 0], poly_arr[:, 1])
                
    # Remove axes
    plt.axis('off')
    
    # Ensure the layout is correct before saving
    plt.tight_layout()
    
    dir_path = '/data/tii/data/nuscenes_maps/nuscenes_trainval_maps_png/'
    os.makedirs(dir_path, exist_ok=True)
    full_path = os.path.join(dir_path, sc)
    new_file_path = full_path.replace('.pkl', '.png')
    
    #print("Figure size before saving:", fig.get_size_inches())
    plt.savefig(new_file_path, bbox_inches='tight', pad_inches=0, transparent=False, format='png')

    print(f'Map of the scenario {sc} saved in png format!')
    plt.close()

###### Map features to npy files

In [ ]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/tii/data/nuscenes_trainval_pkl/')

for sc in scenario_ids:
    scenario = read_scenario(dataset_path='/data/tii/data/nuscenes_trainval_pkl/', mapping=mapping, scenario_file_name=sc) 
    poly_list = []   
    for k in scenario['map_features'].keys():
        for kk in scenario['map_features'][k].keys():
            if kk in ['polyline', 'polygon']:
                poly_arr = scenario['map_features'][k][kk]
                poly_list.append(poly_arr[:, :2])
                #print('map feature shape:', poly_list.shape)
    poly_list = np.concatenate(poly_list, axis=0)
    print('scene map features shape:', poly_list.shape)
    
    dir_path = '/data/tii/data/nuscenes_maps/nuscenes_trainval_maps_npy/'
    os.makedirs(dir_path, exist_ok=True)
    full_path = os.path.join(dir_path, sc)
    new_file_path = full_path.replace('.pkl', '.npy')
    np.save(new_file_path, poly_list)
    print(f'Map of the scenario {sc} saved in npy format!') 


### Waymo

##### Agent features

In [24]:
import numpy as np
import os 
from scenarionet import read_dataset_summary, read_scenario


dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/waymo_converted_556/')

list_seq_length, list_num_agents = [], []
for sc in scenario_ids:
    scenario = read_scenario(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/waymo_converted_556/', mapping=mapping, scenario_file_name=sc)
    seq_length = scenario['length']
    
    list_seq_length.append(seq_length)
    print('seq_length: ', seq_length)
    
    #print('shape of ts: ', scenario['metadata']['ts'].shape)

    scenario_list = []
    for agent in scenario['tracks'].keys():
        if scenario['tracks'][agent]['type'] in ['PEDESTRIAN', 'CYCLIST', 'VEHICLE']:
            state_dict = scenario['tracks'][agent]['state']
            
            #ts = scenario['metadata']['ts']  #-- array of shape (seq_length,)
            vd = state_dict['valid']         #-- array of shape (seq_length,) (either False or True)
            vd = vd.astype(float)            #-- array of shape (seq_length,) (either 0. or 1.) ==> to match nuscenes vd type 
            p  = state_dict['position']      #-- array of shape (seq_length, 3)
            hd = state_dict['heading']       #-- array of shape (seq_length,)
            v  = state_dict['velocity']      #-- array of shape (seq_length, 2)
            h  = state_dict['height']        #-- array of shape (seq_length,)
            w  = state_dict['width']         #-- array of shape (seq_length,)
            l  = state_dict['length']        #-- array of shape (seq_length,)
            
            agent_list = []
            for s in range(seq_length):
                agent_list.append([vd[s], p[s][0], p[s][1], hd[s], v[s][0], v[s][1], w[s], l[s]])
                #agent_list.append([vd[s], p[s][0], p[s][1], p[s][2], hd[s], v[s][0], v[s][1], h[s], w[s], l[s]])
                #agent_list.append([vd[s], ts[s], p[s][0], p[s][1], p[s][2], hd[s], v[s][0], v[s][1], h[s], w[s], l[s]])
            
            scenario_list.append(agent_list)
    
    list_num_agents.append(len(scenario_list))

    scenario_npy = np.array(scenario_list)
    print('before slicing', scenario_npy.shape)
    # truncate the numpy array to the min seq_length: 91 to have a uniform shape in dim 1 and 2
    scenario_npy = scenario_npy[:, :91, :]
    print('after slicing', scenario_npy.shape)

    dir_path = '/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/waymo_556_npy/'
    os.makedirs(dir_path, exist_ok=True)
    full_path = os.path.join(dir_path, sc)
    new_file_path = full_path.replace('.pkl', '.npy')
    np.save(new_file_path, scenario_npy)
    print(f'Scenario {sc} saved !')

print('max seq_length: ', max(list_seq_length))
print('min seq_length: ', min(list_seq_length))
print('max number of agents: ', max(list_num_agents))
print('min number of agents: ', min(list_num_agents))


seq_length:  91
before slicing (55, 91, 8)
after slicing (55, 91, 8)
Scenario sd_waymo_v1.2_4b60f9400a30ceaf.pkl saved !
seq_length:  91
before slicing (58, 91, 8)
after slicing (58, 91, 8)
Scenario sd_waymo_v1.2_4014ae5bcda2726f.pkl saved !
seq_length:  91
before slicing (45, 91, 8)
after slicing (45, 91, 8)
Scenario sd_waymo_v1.2_4a2ef30000d19d90.pkl saved !
seq_length:  91
before slicing (94, 91, 8)
after slicing (94, 91, 8)
Scenario sd_waymo_v1.2_e52c6a9366981ad.pkl saved !
seq_length:  91
before slicing (81, 91, 8)
after slicing (81, 91, 8)
Scenario sd_waymo_v1.2_db4edc9bd0c9d18c.pkl saved !
seq_length:  91
before slicing (32, 91, 8)
after slicing (32, 91, 8)
Scenario sd_waymo_v1.2_9fffe68876965f2e.pkl saved !
seq_length:  91
before slicing (22, 91, 8)
after slicing (22, 91, 8)
Scenario sd_waymo_v1.2_9ea216a54ee07b49.pkl saved !
seq_length:  91
before slicing (55, 91, 8)
after slicing (55, 91, 8)
Scenario sd_waymo_v1.2_2e41fe6faf5cd2ea.pkl saved !
seq_length:  91
before slicing (9

##### Map features

In [25]:
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/waymo_converted_556/')

scenario = read_scenario(dataset_path='/data/ahmed.ghorbel/workdir/autod/unitraj/mdsn/dataset/waymo_converted_556/', mapping=mapping, scenario_file_name='sd_waymo_v1.2_1a7c62ecf73896d8.pkl')

print(scenario['map_features'].keys())
print(len(scenario['map_features'].keys()))

dict_keys(['1', '3', '5', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '24', '30', '31', '33', '34', '35', '37', '38', '39', '40', '42', '43', '44', '45', '46', '47', '48', '49', '50', '52', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '99', '102', '103', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '123', '124', '125', '127', '136', '138', '140', '144', '145', '146', '147', '148', '149', '150', '154', '156', '159', '161', '162', '165', '166', '167', '168', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205',